In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import csv

### GGF HH production

In [ ]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13000.),
                         PythiaParameters = cms.PSet(
        pythia8CommonSettingsBlock,
        pythia8CP5SettingsBlock,
        pythia8PSweightsSettingsBlock,
        parameterSets = cms.vstring('pythia8CommonSettings',
                                    'pythia8CP5Settings',
                                    'pythia8PSweightsSettings'
                                    )
        )
                         )

ProductionFilterSequence = cms.Sequence(generator)
"""

In [ ]:
def fragmentsDictCreator ():
    dict = { '2016': pythia_fragment_CP5,
             '2016APV': pythia_fragment_CP5,
             '2017': pythia_fragment_CP5,
             '2018': pythia_fragment_CP5
                    }
    return dict

In [ ]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    generateConcurrently = cms.untracked.bool(False),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [ ]:
years = ["2016", "2016APV", "2017", "2018"]
mass_point = ["1700"]
tot_events = [300000]*10
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [ ]:
for year in range(len(years)):
    t_example1 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/PairVLQ_x53x53_tWtW_narrow/PairVLQ_x53x53_tWtW_narrow_RH_M1000'
    t_datasetname_year = 'PairVLQ_x53x53_tWtW_narrow_RH_M{massX}_TuneCP5_13TeV-madgraph-pythia8'
    t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/2017/13TeV/madgraph/V5_2.6.5/PairVLQ_x53x53/tWtW/PairVLQ_x53x53_tWtW_narrow_RH_M{massX}_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_ggf_year = "gp_ggf_" + years[year]
    gp_ggf_year = []
    

    #gp_ggf_year = "gp_ggf_" + years[year]
    dataset_names_year = []
    gp_ggf_year = []

    for massX in mass_point:
        d = {"massX": massX}
        tmp_example_dict = t_example1
        gp_ggf_year.append(t_gp_year.format_map(d))
        dataset_names_year.append(t_datasetname_year.format_map(d))
        tmp_dataset_dict = dataset_names_year
        tmp_gridpack_dict = gp_ggf_year
        
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict

## .csv Production

In [ ]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
#tmp_process_pythia_map = {}
#for process in processes:
process_pythia_map = fragmentsDictCreator()
#process_pythia_map.update(tmp_process_pythia_map)

In [ ]:
#for year in range(len(years)):
for year in gridpacks_dict:
    #print(year)
    with open('X53tW'+'_'+year+'_additional.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                        quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator','mcdbid','time','size'])
        #if year == "2016":
        #    pr = 5706
        #if year == "2016APV":
        #    pr = 5730
        #if year == "2017":
        #    pr = 5160
        #if year == "2018":
         #   pr = 5489
            
        k=0
        for i in range(len(mass_point)):
            tmp_fragment=""
            #print(gridpacks_dict[year][i])
            #print(dataset_names[year][i])
            #version = MadgraphVersion.search(path).group(0)
            version="2.6.5"
            dataset_name = dataset_names[year][k]
            if year == "2016APV":
                events = round(tot_events[i]*0.54)
                #prepid = 'B2G-RunIISummer20UL16wmLHEGENAPV-0' + str(pr)
            elif year == "2016":
                events = round(tot_events[i]*0.46)
                #prepid = 'B2G-RunIISummer20UL16wmLHEGEN-0' + str(pr)
            elif year == "2017":
                events = round(tot_events[i])
                #prepid = 'B2G-RunIISummer20UL17wmLHEGEN-0' + str(pr)
            elif year == "2018":
                events = round(tot_events[i])
                #prepid = 'B2G-RunIISummer20UL18wmLHEGEN-0' + str(pr)
                    
            tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][k]) + '\n' + process_pythia_map[year]
            final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year])
            note = dataset_name.replace('_',' ')
            generators="Madgraph_" + version + "  Pythia8"
            mcdb_id = '0'
            time = '2'
            size = '133'
            csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
            k=k+1
            #pr = pr+1